In [1]:
import pandas as pd
import numpy as np
import os
import re

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/IW_student/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
long_ingr=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/sub_long_ingr_data/long_ingr_ge_003.csv',index_col=0)
long_ingr.shape

(1849, 5)

In [3]:
long_ingr.drop('en_ingr',axis=1,inplace=True)

In [4]:
long_ingr=long_ingr.assign(ingredient_name=long_ingr['ingredient_name'].str.split(',')).explode('ingredient_name')

In [5]:
# load data frome here!!!
long_ingr.reset_index(drop=True,inplace=True)

In [6]:
long_ingr=long_ingr[long_ingr['ingredient_name'].str.split().str.len()>3]
long_ingr.shape

(802, 4)

In [27]:
long_ingr

,recipe_id,recipe_href,name,ingredient_name,ingredient
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse auch mitfeta käse,
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade mit nuss,
2,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce knorr,
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur knorr,
4,21993,/rezept/459233/Eierkuchen-Hoernchen.html,hauskaninchen fleisch frisch gegart,hauskaninchen fleisch gegart,
...,...,...,...,...,...
742,51195,/rezept/451254/Raclette-Grillkartoffeln-mit-Sc...,kräuterbutter café deparis. z.b.- aus meinem m...,kräuterbutter café deparis. z.b.,
743,113827,/rezept/444419/Auberginen-2-Dieter-s-Art.html,aglio olio de peperoncino,aglio olio de peperoncino,
744,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel fürs gemüse fastvergessen,
745,92957,/rezept/370498/Krokantpudding-Kirsch-Streuselk...,vanilleback von dr. oe,vanilleback dr. oe,


In [7]:
long_ingr.reset_index(drop=True,inplace=True)

In [8]:
long_ingr_list=list(set(long_ingr['ingredient_name'].values.tolist()))
len(long_ingr_list)

758

In [30]:
long_ingr_list.sort()
long_ingr_list

[' kuchenteig a',
 ' pellkartoffel mitrucolapesto',
 '1 msp sambal oelek',
 '1 tl sambal oelek',
 '16 l wasser od. suppe',
 '200 gr. jenach appetit',
 '5 gewürzepulver bamboo garden',
 'abgeriebende schale 1orange',
 'abgeriebeneschale 2 unbehandeltenorangen',
 'afrikanischer peppersgefüllt mit frischkäsemango',
 'aglio olio de peperoncino',
 'aglio olio e peperoncino',
 'alpro soja drink kalzium',
 'alternativdas mark einer ausgekratztenvanilleschote',
 'amarenakirschen dem glasmit saft',
 'anis man kan auchandere gewürze nehmen',
 'aromatische äpfel z.b.roter boskop',
 'asia gemüse zbiglo',
 'asia wokgemüse tk iglo',
 'asiatische sweet and soursauce',
 'aufleger ausschokolade den selbenanbietern',
 'backmischungschokoflockenkuchen vomdem onkel bielefeld',
 'backpulver 300 gr mehl',
 'backpulver u.eine salz',
 'baking powder muss sein',
 'bambussprossen dosewer mag',
 'basis f. saucebolognese“ barilla',
 'batida de coco kokoslikör',
 'batida de coco likör',
 'bauchspeck mariniert aufr

In [9]:
# whole delete
whole_delete=['drehe diestemme vorsichtig raus','edelgard bitte verzeih mir', 'ein mittleres sieb war knappgefüllt',
'eine kleine schoote farbeegal','eine tiefe pfanne mitdeckel','ergibt einen schönenglänzenden überzug',
'feinkörnig damitder kuchen etwas crunchywird', 'ferrero rocher als deko', 'fertige sternchenweihnachtdeko von ruf',
'feuerfeste schälchen à hoch','filoteig blätter 30 x','fleckerl sindkleine quadrate ausnudelteig',
'frisch2c kann aberauch gefrohrene sein','fuer das baba ghanoush','fächerdie halbe masse vom botenzubereiten',
'geht auchjedes andere fleisch','ggf. ohneden harten edelschimmel rand','halt nachgeschmack menge wählen',
'ihnin ein netz ziehen lassen','ist der weinin dem man sein fleischgetaucht hat','kalte holzasche demkamin buche',
'kann sie auchfertig im supermarkt kaufen','kleine rosa herzen zur dekofür mein herzchen',
'kleine spritze zumeinfüllen2c kaiser','kochen mit finess 7% von dudarfst','kochen mitfiness von du darfst',
'konfetti zuckerdeko von rewe','kristallzucker rot gefärbtfür die deko der gläser','kuvertüre je 50% vonbitter',
'lauwarmes wasser .. mengesiehe rezeptbeschreibung','leichtmehlig kochend z.b secura','lotte man sagt auch seeteufel',
'mixrustikal von hammer mühle','muffinförmchen papier2cimmer zwei pro küchlein','pck. je 25 gtk',
'pop rocks dieseknallkügelchen 2c','pro person 125 grroh gewogen','röschen vom t´e`te demoine',
'saiten wie wir imschwäbischen sagen','sekt trocken gut qualität','sossenbinder um diesosse sämig',
'speisestärke inkaltem wasser auflösen','tasse mit inhaltals mass','triple sec von bols',
'um das kirschwassergleichmässig auf demtortenboden','von der füllung fürtupfer auf der torte',
'wasser2cleicht er als einliter','weisse kuvertüre zumverschliessen der kugeln','weitere gewürze nach artdes salates',
'wird erstkurz vor dem servierenbenötigt']

In [10]:
long_ingr=long_ingr[~long_ingr['ingredient_name'].isin(whole_delete)]

In [11]:
long_ingr.reset_index(drop=True,inplace=True)

In [ ]:
# ^z.b, ^z. b.,

In [12]:
for index,row in long_ingr.iterrows():
    if re.search(r'^z.b',row.ingredient_name)!=None:
        #print(row.ingredient_name)
        long_ingr.drop(index,inplace=True)
long_ingr.reset_index(drop=True,inplace=True)

In [13]:
for index,row in long_ingr.iterrows():
    if re.search(r'^z. b',row.ingredient_name)!=None:
        #print(row.ingredient_name)
        long_ingr.drop(index,inplace=True)
long_ingr.reset_index(drop=True,inplace=True)

In [14]:
# delete word
delete_words=['2 tl','untermischen','200gr.','6 blatt','bitte','nur','gehackt','gruen','fürdeko','dr. oetker','einige blätter',
'einpaar','es geht','festes','flaschen guinnes','von knorr','frische','dicke','deutsche','instant','galetta',
'gefüllte','ganze','stifte','frisch','eigeneherstellung','ersatzweisegekaufte','auchschlachter','gepökelter',
'vomschwäbisch','geriebenen','geröstete','der mühle','gefroren','zbsp','hackepeterfertig',
'gewürzt','übrig','gedünstet','heissemwasser','kleine','deko','klösse','von','demgrün','getränk','69 kcal',
'420 gabtropfgewicht','umfeuchtigkeit','aufzunehmen','prise','von dr.oetker','vonz.b. knorr','frischvom','geflügelhof',
'reichlich','hautabziehen u.','gespritzt','überlassen','vom','hier','sagtman','extra','leicht','fixitalienische','art',
'gut','abgetropft','stichfest','ohneknochen','duroc','sind','auchhalbierte','für1','fridabella','nachgeschmack',
'spritzer','ider','gewürfelt','sorrynicht','gefüllt','siehe','foto','türkisches','geht','genauso','wie','der',
'a`250','gramm','1prise','bei bedarf','grob','gewürfelt','von maggimeiserklasse','tiefgefroren','gemahlener',
'gemahlen','aufgefangenes','dergetrockneten','backfertige','backfertiger','natürlich','diefrischen','beisaison...',
'fettarm','bio','geschälte','geraspelte','ohnestein']

In [15]:
def del_words(ingredient):
    no_words=' '.join([c for c in ingredient.split() if c not in delete_words])
    return no_words

In [16]:
long_ingr['ingredient_name']=long_ingr['ingredient_name'].apply(lambda x:del_words(x))

In [17]:
# delte part
delete_parts=['u.eine prise','der dosewer mag','mariniert aufrussische art','betty bossi','sehr scharf','habweisswein genommen',
'wir hatten','aus meinem kochbuch','evtl. noch eigenemgeschmack']

In [18]:
for part in delete_parts:
    for index, row in long_ingr.iterrows():
        if re.search(part,row.ingredient_name)!=None:
            long_ingr.loc[index,'ingredient_name']=re.sub(part,'',row.ingredient_name)

In [19]:
# delete after
delete_after=['man kan.*','vom.*','muss.*','alternativ.*','was man.*','deko.*','ich.*','hatte.*',
              'wer.*','nach wahl.*','wird.*']

In [20]:
for after in delete_after:
    for index, row in long_ingr.iterrows():
         if re.search(after,row.ingredient_name)!=None:
                long_ingr.loc[index,'ingredient_name']=re.sub(after,'',row.ingredient_name) 

In [21]:
# parse ingredient
parse_ingredients=['spekulatius','zitronensaft','kalb filets','lauch stange',' staudensellerie','stevia zucker','stracciatella',
                   'streuselteig','suppenfleisch rind','suppengrün','suppennudeln','sweet chili sauce','säuerliche äpfel',
                   'himbeer balsam','steviazucker','tete de moine','tintenfisch','toastkäse','tomate al gusto',
                   'tomatenmark','tomatenpaste','tomatensuppe','tomatenöl','tortencreme','trauben','frostgemüse',
                   'kalbgulasch','schwarzerpfeffer','vanilleback','vanillepudding','vanilleschote','zebraröllchen',
                   'vollmilchschokolade','olivenöl dertomaten','gezupftekräuter','waldpilz sauce','gekörnterbrühe',
                   'diehälfte rotwein','weintraube weiss','weisse backschokolade','weisser kandis','weisser pfeffer',
                   'weizen mehl','weizenmehl','welsfilet','wiener tortenboden','winter apfel getränk','woknudeln','würstchen',
                   'würzmischung','waffeln','ziehbutter2c','zitronenschale','zwetschgen','paprikapulver','äpfel boskop',
                   'äpfel frisch','äpfel granny','überbackene tomaten','sambal oelek','5 gewürzepulver',
                   'orange','vanilleschote','äpfel','asia gemüse','asia wokgemüse','croissants','backpulver',
                   'baking powder','bambussprossen','bauchspeck','batida de coco','himbeeren','basilikum',
                   'bohnen weiss','bohnensalat','bratwuerste','cherrytomaten','currypaste','datteltomaten','erdbeere',
                   'mandarinen','creme fraiche','vanillin zucker','vanillezucker','zwiebel','emmentaler','enten',
                   'erdnussriegel','sojamehl','knoblauch','fenchelsauce','grillhähnchen','vanille bourbon',
                   'fleischbrühe','fleischsalat','fleischwurst','aromazitrone','fonduefett','tomatensauce',
                   'typ 550 mehl','ingwer','parmesan','saison dill','petersilie','ravioli','prik kee noo',
                   'frischkäse','frühlingszwiebel','fruchtsaft','garnelen','crema dibalsamico','geflüge','frühlingszwiebel',
                   'kartoffeln','gelbe u. grüne erbsen','orangenchili','thymian','gemüse der saison','gemüsevorrat',
                   'schwarze oliven','muskat','gewürzgurken','gewürzmischung','pimentkörner','pfirsiche','glühwein',
                   'gouda käse','grana padano parmesan','trüffel','weinhappen','bratwürste','salatblätter','bier',
                   'türkischeweisskohlköpfe','grünen salat',' kulturlöwenzahn','gruyère','grüne bohnen','grüne oliven',
                   'grüne pfeffer','grüne stangenbohnen','estragon','bildpaprikaknacker','gyrosspiesse','götterspeise himbeer',
                   'götterspeise kirsch','zucchinis','hackfleisch','kandierte kirschen','ziegenkäse','haselnusskerne',
                   'haselnüsse','hauskaninchen','meersalz','helle sosse','helles bier','herzkirschen','spitzpaprika','honig',
                   'hirsch','holländische sosse','hähnchenbrustfilet','hüftsteaks','hühnerbrühe','gewürzsalz','ingwer frisch',
                   'ital. speck','italienische kräuter','italienische kräuter','der pute','vanilleeis','semmelbrösel',
                   'steviaprodukt','joghurt natur','johannisbeergelee','kabeljaurückenfilet','kalbsbrust','kalbskotelett',
                   'kalte sahne','feinemblattspinat','kartoffelwurst','kasseler','barbarieente','kirschen frisch','kirschsaft',
                   'geflügelbratwürstegrillminis','vermicelli','weisse bohnen','knoblauchpaste','knoblauchpulver',
                   'chilli con carne','kochpuddingpulver mitschokoladengeschmack','kokossahne2c','korianderpulver',
                   'kroketten','krustebrot','kräuter der saison','kräuter frisch','kräuteressig','küchenfertigeshähnchen',
                   'grünen oliven','lammkeule','lammkoteletts','lammlachs','lauchherzen','lauchzwiebel',
                   'leberkäse','pizzateig','einerprise salz','limandafilet','lime juice','limoncello','linsen braun',
                   'kaffee caffecrema','madeira rot','mandelmehl','mandeln süss','mangosaft','heringe','marzipan','kräuteröl',
                   'mexikanischepfannengemüse','milchschokolade','curry powder','milde pepperoni','pflaumentomaten','mohnfüllung',
                   'multivitaminsaft','mürbeteig','nacken vom schwein','rotepfefferbeeren','olivenöl','nudelteig','ofenkartoffel',
                   'orangen frisch','orangene mini','palmzucker','fischfilet','parboild reis','peperoni grün','peperoni mix',
                   'peperoni rot','pepperoni rot','pfirsicheis','pilzmehl','pilzsosse','pimientos','pinkelwürste','pommes frites',
                   'porree','prise zucker','mon cherie','prostrudel','puder zucker','putenfilet','putenfleisch','putenschinken',
                   'cremefine','rapsöl','ras el hanout','rauchfleisch','rauchkäs','rawit','raz eö hanout','remoulade','milchreis',
                   'ribeye steak','ribiselmarmelade','riesen frankfurter','riesen garnelen','rinderfilet','rinderschulter','rindfleisch',
                   'roasted pork','roasted turkey','rote bete','rote chili','rote paprika','rotwein','mayonaise','salami','salatblätter',
                   'schafskäsewürfel','schnittkäase','schinkenstreifen','schlagsahne','schlemmer creme','weissweinz.b.',
                   'balsamico essig','schwein nacken','schwein speck','schweinebauch','weissenspargel','scotch bonnet chilie',
                   'rippe palmin','slibowitz','sour cream','spaghetti','spanferkelkeule','spargel weiss','spargelschalen']

In [22]:
for ingredient in parse_ingredients:
    for index,row in long_ingr.iterrows():
        if re.search(ingredient,row.ingredient_name)!=None:
            #print(row.ingredient_name)
            long_ingr.loc[index,'ingredient_name']=ingredient

In [ ]:
# replace 'mit' with ','

In [23]:
long_ingr['ingredient_name']=long_ingr['ingredient_name'].apply(lambda x:x.replace('mit',','))

In [24]:
long_ingr

,recipe_id,recipe_href,name,ingredient_name
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,"weisse schokolade , nuss"
2,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur knorr
4,21993,/rezept/459233/Eierkuchen-Hoernchen.html,hauskaninchen fleisch frisch gegart,hauskaninchen
...,...,...,...,...
742,51195,/rezept/451254/Raclette-Grillkartoffeln-mit-Sc...,kräuterbutter café deparis. z.b.- aus meinem m...,kräuterbutter café deparis. z.b.
743,113827,/rezept/444419/Auberginen-2-Dieter-s-Art.html,aglio olio de peperoncino,aglio olio de peperoncino
744,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel
745,92957,/rezept/370498/Krokantpudding-Kirsch-Streuselk...,vanilleback von dr. oe,vanilleback


In [25]:
long_ingr=long_ingr.assign(ingredient_name=long_ingr['ingredient_name'].str.split(',')).explode('ingredient_name')

In [26]:
long_ingr=long_ingr[long_ingr['ingredient_name']!='']
long_ingr['ingredient_name']=long_ingr['ingredient_name'].str.strip()
long_ingr.reset_index(drop=True,inplace=True)

In [27]:
long_ingrs=list(set(long_ingr['ingredient_name'].values.tolist()))
len(long_ingrs)

603

In [28]:
long_ingrs.sort()
long_ingrs

['16 l wasser od. suppe',
 '200 gr. jenach appetit',
 '5 gewürzepulver',
 'aglio olio de peperoncino',
 'aglio olio e peperoncino',
 'alpro soja drink kalzium',
 'amarenakirschen dem glas',
 'anis',
 'aromazitrone',
 'asia gemüse',
 'asia wokgemüse',
 'asiatische sweet and soursauce',
 'aufleger ausschokolade den selbenanbietern',
 'backmischungschokoflockenkuchen',
 'backpulver',
 'bacon ananasu. gouda',
 'baking powder',
 'balsamessigaus modena',
 'balsamico essig',
 'bambussprossen',
 'barbarieente',
 'basilikum',
 'basis f. saucebolognese“ barilla',
 'batida de coco',
 'bauchspeck',
 'bavette no. 13 barilla',
 'bayrischer we',
 'belegt. es darf auch gerner sein belag sein',
 'bier',
 'bildpaprikaknacker',
 'biskuit tortenboden',
 'bisquitboden 2 eiern',
 'bissl dill u. 8kräutermischung',
 'bitterschokolade',
 'blattsalat',
 'blattschokolade zumeinstreuen des teigs',
 'blue curacao sirup ohnealkohol',
 'bohnen weiss',
 'bohnensalat',
 'bratwuerste',
 'bratwürste',
 'braune linsen 53

In [29]:
long_ingr.to_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/sub_long_ingr_data/long_long_ingr_done.csv')

In [2]:
long_ingr=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/sub_long_ingr_data/long_long_ingr_done.csv',index_col=0)
long_ingr.shape

(789, 4)

In [3]:
long_ingr

,recipe_id,recipe_href,name,ingredient_name
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade
2,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,nuss
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce
4,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur knorr
...,...,...,...,...
784,113827,/rezept/444419/Auberginen-2-Dieter-s-Art.html,aglio olio de peperoncino,aglio olio de peperoncino
785,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel
786,92957,/rezept/370498/Krokantpudding-Kirsch-Streuselk...,vanilleback von dr. oe,vanilleback
787,104968,/rezept/489228/Salat-Cedro-mit-Bueffelmozzarel...,"wer das nicht hat,nimmt eingutes salz mit kräu...",nimmt eingutes salz


In [4]:
long_ingr[long_ingr['ingredient_name']=="weinhappen"]['name'].values

array(['griechische weinhappen stehen im kb'], dtype=object)

In [ ]:
# change manually

In [4]:
# whole delete
whole_delete=['200 gr. jenach appetit','aufleger ausschokolade den selbenanbietern','belegt. es darf auch gerner sein belag sein',
'bunte back streusel','castel franco la rossa','diabetiker nehmen stattpuderzucker flüssigensüssstoff',
'dinkel reis2c ersatzweiseebly weizen','essbare blüten zur','mind anteil','mind. gehalt','pizza tomatensosse wennkeine sosse']

In [5]:
long_ingr=long_ingr[~long_ingr['ingredient_name'].isin(whole_delete)]
long_ingr.reset_index(drop=True,inplace=True)

In [6]:
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='woknudeln']='noodle'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='worcestersosse salzpfefferaus mühle']='worcestershiresauce,salt,pepper'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='worcestershiresauce u einigespritzer tabasco']='worcestershiresauce,tabasco'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='woknudeln']='noodle'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='türkischeweisskohlköpfe']='cabbage'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='wasser senf salz pfefferessig']='wasser,senf,salz,pfeffer,essig'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='türkischeweisskohlköpfe']='cabbage'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='thai lemon pepper seasoningoil']='Thai_lemon_pepper_oil'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='thai kitchen nest noodles']='noodle'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='thai hot green curry dergewürzmühle']='curry'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='tete de moine']='cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='teil alkohol mon cherioben 6 stk']='Mon_Cheri'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='steviazucker']='stevia'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='steviaprodukt']='stevia'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='stevia zucker']='stevia'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='st. schnittkäse halbfestmini babybel']='cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='sriracha hot chilli sauce']='chili_sauce'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='spitzpaprika']='chili'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='semola di grano duro']='wheat'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='scotch bonnet chilie']='bell_pepper'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='scharfes messer zumzerkleinern schokolade']='schokolade'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='scharfer senf bismegascharfer senf']='senf'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='saure sahne o. schmand']='saure sahne'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='sambal ala bankok gibtsbeim asiaten']='sambal oelek'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='salz pfeffer']='salt,pepper'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='salz essig maggi wasser mehl']='salt,vinegar,water,flour'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='sal de flor rosmarins']='salt,rosemary'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='saft zumbeträufeln des vogels']='juice'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='saft sauerkirschen essind']='cherry_juice'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='rote prik ke noo chili']='chili'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='rollen harzer käse je']='cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='rispen cherry dattel tomaten']='cherry_tomato'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='riesen frankfurter']='sausage'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='reste garganelli']='pasta,chili_sausse'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='red chili coconut sosse']='chili,coconut_sauce'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='rauchkäs']='cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='puderzucker u kakaopulverzum bestäuben']='sugar,coco_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='prik kee noo']='chili'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='piment la vera picante']='Piment_la_Vera_picante'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pilze champignon osteinpilz']='mushroom'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='philadelphia 3 pakete a']='cream_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pfirs']='peach'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pfeffer... salz.... paprikaedelsüss... china']='pfeffer,salz,paprika_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pfeffer u. salz']='pfeffer,salz'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pfeffer etwas kräutersalz']='pepper,kräutersalz'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pfeffer 2 gehäufte tlpaprika edelsüss']='pepper,paprika_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='bayrischer we']='weichkäse'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='creme fra']='creme fraiche'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='curry dragon']='curry'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='curry... paprikaedelsüss... pfeffer... salz']='curry,paprikaedelsüss,pfeffer,salz'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='curryananas frischkasefüllung']='curry,pineapple,cream_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='dany sahne nuss nougatgeschmack']='nuts_nougat_flavored_cream'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='dass cocktailsaucewegen scampis sehrpassend ist']='cocktail_sauce,scampi'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='curry dragon']='espresso'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='curry dragon']='joghurt'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='dinkel mehl weiss glatt']='flour'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='eierlikör']='egg_liqueur'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='einen biskuitboden 4eiern']='biskuit,egg'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='eis amstiel']='vanilla_ice_cream'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='emmentaler']='emmentaler_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='gekörnterbrühe']='broth_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='gläser babybrei karotte a']='baby carrot porridge'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='gorgonzola dolce e cremoso']='gorgonzola_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='grand vine de bordeausauternes']='wine'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='gulasch halb schwein halbrind']='pork,beef'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='hüftsteaks']='sirloin_steak'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='ing']='ginger'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='knacker od. 2 kl']='cracker'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='käse z.b edammer od gouda']='gouda_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='käse z.b. gouda geraspelt']='gouda_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='mexican hot rice mix']='Spicy_Mexican_Rice'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='mexikanischepfannengemüse']='Mexikanisches_Pfannengemüse'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='mousse au chocolat sterne']='Mousse_au_Chocolat_Sterne'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='mousse au chocolat2c 2anbieter im leb. handel']='Mousse_au_Chocolat_Sterne'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='o']='Tomaten_Passiert'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='passierte tomaten']='Tomaten_Passiert'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='ofenkartoffel']='potato'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='paprika rot2c gelb2c grün2c']='bell_pepper'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='parboild reis']='rice'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pecorino su sardo gerieben']='cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pesto verde bertolli']='Pesto_Verde'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/pyt

In [7]:
# part delete
delete_parts=['dem glas','aus modena','zumeinstreuen des teigs','ohnealkohol','530 gr.abtropfgew','fa. aldi süd',
'brotrinde 2','wein zumsteak','getrocknet','scharf süss vonlien ying','evtl. fa.fuchs','meggle','iglo','es sindbei uns immer 2',
'einerprise','liqueur','einen anderenkräftigen','verwenden','einige kräftige spritzer','bei kochende','einige',
'el meiner','auf vorrat','siehehähnchenspiess','noch vonmeiner mutter hergestellt','bestäubt','tk','farmers',
'feinemblatt','feiner','gefr','markt','derknolle gelösst','filet','1 meinkochbuch','flying goose','fleischstückchen vonpo',
'geriebene schale','geriebener','geriebenz schaleeiner','getrocknete','sindeigene herst','bei derzubereitung genannt',
'nach gusto','gezupfte','grillen übriggebliebene','stangen','meinemgarten','gute laune','zubereitung','spiesse',
'herbe','z. b.milka','natur','filets','kalte','neo lidl','gemörsert','im winterkompot','knochen','knorr',
'zumstreuen gartenkräuter','2c','feingehackt eigene ernte','dem kühlregal','im frischepack','küchenfertiges',
'milde','etc. als einlage dazunehmen','nimmt eingutes','nach meinem rezeptbandnudeln','z.b. hörnchenal dente',
'dem glasfür den farbtupfer','entkerntenthäutet kleingeschnitten','s.rezept im kochbuch','metzgerihre vertrauens 2c',
'dr. oetker','paula','bzw. meine pikantemischung','pralinen hohlkörper2chandelsware 2 anbieter iminternet2c wohlers',
'dr.oetker','5 kilo','rand','kochpaste scharf','gegart körnige sorte','nach anleitug kochen','4 zweige',
'demeigenen garten','s. mein kochbuch','über den pumpernickelgiessen','kochen herdnehmen','weihenstephan nur50%f fett',
'saison','gemischt nach pers.plan','properson etwa 150','# indonesischechilipaste','nach belieben2c wirfanden',
'kleingeschnitten je','2c das istungefähr 1','onkelbens','würfel','streifen','eventuell als klecksauf den servierten eintopf',
'ist dasmass','habe milka genommen','milka crunchicaramel zerkleinert','milka fire','streusel als',
'selbstwürzen','vonsojatoo','z.b. uncle benssüss','gegart','don enrico','the cheech','gehäutet u','im blendepuerriert',
'konserve geschält','rot geachtelt','3 lagen','dem kb vonkassy','dunkel 2 teilig','meinrezept einfachentortenboden',
'nachanweisung herstellen','typ 550','uncle bens','vondr.öttker','warmen......','gebacken','evt.',
'250 gr. altengoude','per 1 l saft','per 1 liter saft','je nach phase','überbackene']


In [8]:
for part in delete_parts:
    for index,row in long_ingr.iterrows():
        if re.search(part,row.ingredient_name)!=None:
            long_ingr.loc[index,'ingredient_name']=re.sub(part,'',row.ingredient_name)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
# keep
#'^\bmilch\b'
for index,row in long_ingr.iterrows():
    if re.search(r'^\bmilch\b',row.ingredient_name)!=None:
        #print(row.ingredient_name)
        long_ingr.loc[index,'ingredient_name']='milch'

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
long_ingr=long_ingr.assign(ingredient_name=long_ingr['ingredient_name'].str.split(',')).explode('ingredient_name')

In [11]:
long_ingr

,recipe_id,recipe_href,name,ingredient_name
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade
2,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,nuss
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce
4,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur
...,...,...,...,...
772,113827,/rezept/444419/Auberginen-2-Dieter-s-Art.html,aglio olio de peperoncino,aglio olio de peperoncino
773,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel
774,92957,/rezept/370498/Krokantpudding-Kirsch-Streuselk...,vanilleback von dr. oe,vanilleback
775,104968,/rezept/489228/Salat-Cedro-mit-Bueffelmozzarel...,"wer das nicht hat,nimmt eingutes salz mit kräu...",salz


In [24]:
long_ingr=long_ingr[long_ingr['ingredient_name']!='']
long_ingr['ingredient_name']=long_ingr['ingredient_name'].str.strip()
long_ingr.reset_index(drop=True,inplace=True)
long_ingr.shape

(774, 4)

In [26]:
long_ingrs=list(set(long_ingr['ingredient_name'].values.tolist()))
len(long_ingrs)

505

In [27]:
long_ingrs.sort()
long_ingrs

['5 gewürzepulver',
 '8 kräuter',
 'amarenakirschen',
 'anis',
 'apple',
 'aromazitrone',
 'asia gemüse',
 'asia wokgemüse',
 'asiatische sweet and soursauce',
 'baby carrot porridge',
 'backmischungschokoflockenkuchen',
 'backpulver',
 'bacon ananasu. gouda',
 'baking powder',
 'balsamessig',
 'balsamico essig',
 'bambussprossen',
 'barbarieente',
 'basilikum',
 'batida de coco',
 'bauchspeck',
 'beef',
 'bell_pepper',
 'bier',
 'bildpaprikaknacker',
 'biskuit',
 'biskuit tortenboden',
 'bisquitboden 2 eiern',
 'bitterschokolade',
 'blattsalat',
 'blattschokolade',
 'blue curacao sirup',
 'bohnen weiss',
 'bohnensalat',
 'bolognese',
 'bouillon pur huhn',
 'bratwuerste',
 'bratwürste',
 'braune linsen',
 'brotbackmischung volkornbrotder',
 'broth_powder',
 'cabbage',
 'café de paris gewürz',
 'caramel schokoriegel',
 'cheese',
 'cherry_juice',
 'cherry_tomato',
 'cherrytomaten',
 'chili',
 'chili_con_carne',
 'chili_sauce',
 'chili_sausse',
 'chilliflocken',
 'chillisosse',
 'china wü

In [14]:
# whole delete
whole_delete=['ginge aberauch dose ggfpfirs','gärten siamvon herbaria']

In [15]:
long_ingr=long_ingr[~long_ingr['ingredient_name'].isin(whole_delete)]

In [16]:
# delete part
delete_parts=['für den farbtupfer','enerntenthäutet kleingeschnitten','metzgerihre vertrauens',
              'dr. oeer','hohlkörperhandelsware 2 anbieter iminternet wohlers','u 150 .geras.','davon auch densaft']

In [17]:
for part in delete_parts:
    for index,row in long_ingr.iterrows():
        if re.search(part,row.ingredient_name)!=None:
            long_ingr.loc[index,'ingredient_name']=re.sub(part,'',row.ingredient_name)

In [ ]:
# word delete
# 'auch','oder','vondr.öter',

In [18]:
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='alpro soja drink kalzium']='soy_sauce'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='weissweinz.b.']='white_wine'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='säuerl']='apple'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='schnitäase']='cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='sal de flor rosmarin']='salz,rosemary'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='ras el hanout']='Ras_el_hanout'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='pimenörner']='piment'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='paprikaedelsüss']='sweet_paprika_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='paprikaedelsüss alles']='sweet_paprika_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='paprika edelsüss od. scharf']='paprika_powder'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='palmin 100 % kokosfett 1']='Kokosfett'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='oregano. herb de provence']='Herb_de_Provence'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='oregano vonoro di parma']='Pizzasauce_Oregano'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='16 l wasser od. suppe']='water'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='2zitronen']='lemon'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='aglio olio de peperoncino']='aglio_olio_e_peperoncino'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='aglio olio e peperoncino']='aglio_olio_e_peperoncino'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='aglio_olio_e_peperoncino']='Aglio_olio_e_peperoncino'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='egg_']='egg_liqueur'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='basis f. saucebolognese“ barilla']='bolognese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='bavette no. 13 barilla']='pasta'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='bouillon pur gemüse']='8 kräuter'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='bissl dill u. 8kräutermischung']='8 kräuter'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='brunch sweet thai chili']='Brunch_sweet_thai_chili'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='cappucinopuler milkaweisse schokolade jacobs']='coffee'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='chateau de mornag']='Chateau_de_mornag'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='chili  habanero']='habanero_chili'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='chili gelbe habanero']='habanero_chili'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='chilli con carne']='chili_con_carne'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='chilli sauce sweet süssthai']='sweet_chili_sauce'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='gierschkerbel pilzkraut kraut derunsterbl']='petersilie,schnittlauch,rucola,olivenkraut,basilikum'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='götterspeise himbeer']='Götterspeise_raspberry'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='hot pepper sauce chillisauceoä']='chili_sauce'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='ing']='ginger'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='italienischer haräse z.b. grana padano']='Grana_Padano_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='italienischer haräse z.b.grana padano']='Grana_Padano_cheese'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='jamon iberico puro bellota']='Jamon_Iberico_Puro_Bellota'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='kräuter oro di parma']='Tomaten_Passiert_mit_Kräuter'
long_ingr['ingredient_name'][long_ingr['ingredient_name']=='lösl']='salz'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [20]:
long_ingr[long_ingr['ingredient_name']=='säuerl']

,recipe_id,recipe_href,name,ingredient_name
417,20495,/rezept/245905/Apfel-Streuselkuchen.html,säuerliche äpfel z. b.boskop,säuerl
605,11257,/rezept/479347/gefuellte-Supreme-vom-Perlhuhn....,kleine säuerliche äpfelmit roten bäckchen,säuerl


In [19]:
long_ingr['name'][long_ingr['ingredient_name']=='le'].values

array(['eiweiss,leichtaufgeschlagen mit einerprise salz'], dtype=object)

In [22]:
long_ingr[long_ingr['recipe_id']==11257]

,recipe_id,recipe_href,name,ingredient_name
605,11257,/rezept/479347/gefuellte-Supreme-vom-Perlhuhn....,kleine säuerliche äpfelmit roten bäckchen,säuerl


In [20]:
# keep_ingredient
long_ingr_keep=long_ingr[long_ingr['ingredient_name'].str[0].str.isupper()]

In [28]:
long_ingr=long_ingr[~long_ingr['ingredient_name'].str[0].str.isupper()]
long_ingr.shape

(774, 4)

In [ ]:
# translation

In [29]:
%env GOOGLE_APPLICATION_CREDENTIALS =/home/IW_student/work/Ingredeints-translation-1d0aa41c5717.json

env: GOOGLE_APPLICATION_CREDENTIALS=/home/IW_student/work/Ingredeints-translation-1d0aa41c5717.json


In [30]:
from google.cloud import translate_v2 as translate
translate_client = translate.Client() # add environment varibale

In [31]:
def translation(ingredient):
    results=translate_client.translate(ingredient,target_language='en')
    return results['translatedText']

In [32]:
long_ingr['en_ingr']=long_ingr['ingredient_name'].apply(lambda x:translation(x))

In [33]:
long_ingr.reset_index(drop=True,inplace=True)

In [34]:
long_ingr['en_ingr']=long_ingr['en_ingr'].str.lower()

In [79]:
long_ingr

,recipe_id,recipe_href,name,ingredient_name,en_ingr
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse,cream cheese
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade,white chocolate
2,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,nuss,nut
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce,waldpilz sauce
4,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur,pure sauce
...,...,...,...,...,...
769,51195,/rezept/451254/Raclette-Grillkartoffeln-mit-Sc...,kräuterbutter café deparis. z.b.- aus meinem m...,kräuterbutter café deparis. z.b.,herb butter café deparis. e.g.
770,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel,onion
771,92957,/rezept/370498/Krokantpudding-Kirsch-Streuselk...,vanilleback von dr. oe,vanilleback,vanilleback
772,104968,/rezept/489228/Salat-Cedro-mit-Bueffelmozzarel...,"wer das nicht hat,nimmt eingutes salz mit kräu...",salz,salt


In [77]:
long_ingr[long_ingr['en_ingr']=='würzbutter cafe de paris']

,recipe_id,recipe_href,name,ingredient_name,en_ingr
635,60494,/rezept/403142/Rumpsteak-Cafe-de-Paris.html,würzbutter %x22café de paris%x22aus meinem koc...,würzbutter café de paris,würzbutter cafe de paris


In [36]:
long_ingr_en=list(set(long_ingr['en_ingr'].values.tolist()))
len(long_ingr_en)

478

In [37]:
long_ingr_en.sort()
long_ingr_en

['5 seasoning powder',
 '8 herbs',
 'almond flour',
 'amarenakirschen',
 'anis',
 'apple',
 'apples',
 'aromazitrone',
 'asia vegetables',
 'asia wokgemüse',
 'asian sweet and soursauce',
 'asparagus white',
 'at the port',
 'baby carrot porridge',
 'backmischungschokoflockenkuchen',
 'bacon pineapple. gouda',
 'baked tortellini von',
 'baking powder',
 'balsamessig',
 'balsamic cream',
 'balsamic vinegar',
 'bamboo shoots',
 'barbarian bone',
 'basil',
 'bean salad',
 'beans white',
 'beef',
 'beef shoulder',
 'beer',
 'beetroot',
 'beetroot carrots',
 'bell_pepper',
 'belly bacon',
 'biscuit cake base',
 'biscuits',
 'bisquit base 2 eggs',
 'bitter chocolate',
 'black olives',
 'black pepper',
 'blue curacao syrup',
 'bolognese',
 'bourbon finesse',
 'bovine',
 'bratwuerste',
 'brotbackmischung volkornbrotder',
 'broth_powder',
 'brown lentils',
 'cabbage',
 'café de paris spice',
 'cake base biscuit',
 'cake batter a',
 'cake cream',
 'calf breast',
 'calf goulash',
 'caramel chocol

In [80]:
change_dict={'amarenakirschen':'amarena cherry','aromazitrone':'lemon_flavor','asia vegetables':'vegetables',
            'asia wokgemüse':'vegetables','at the port':'limanda','bacon pineapple. gouda':'bacon,pineapple,gouda_cheese',
            'baked tortellini von':'tortellini','balsamessig':'balsamic vinegar','barbarian bone':'duck',
            'beetroot carrots':'beetroot,carrots','belly bacon':'bacon','bisquit base 2 eggs':'biscuit cake base,egg',
            'bratwuerste':'sausage','brotbackmischung volkornbrotder':'whole_wheat_bread','café de paris spice':'Cafe_de_paris',
            'cake base biscuit':'biscuit cake base','cake batter a':'cake batter','cheese geramont':'geramont_cheese',
            'cheese gouda':'gouda_cheese','cherrytomaten':'cherry_tomato','chicken breast':'chicken','chili_sausse':'chili_sauce',
            'chilli pepper':'chili,pepper','chilli with beef':'Chili_con_carne','choko cappucchino typ milkas chokolade':'chocolate',
             'chop':'rib_steak','coconut beat':'Batida_de_coco','cores':'habanero_chili','cream filling':'cream',
            'cream from':'creme fraiche','cucumber kernels 3 lettuce cucumbers':'cucumber',
             'cucumber water from the mustard pickles':'gherkin','dark cake base 3 pieces':'cake_base',
            'dinkelstrudelteig':'strudel dough','dr. original vanillageschmack pudding':'vanilla_pudding',
             'either':'duck','el n vinegar as an egg substitute':'vinegar','endivienkartoffelsalat':'endive_potato_salad',
            'expresspulver':'espresso','feigen balsamic cream':'Crema di balsamico','fenchelpfeifen':'fennel',
             'fenchelsauce':'fennel_sauce','fettuccine with cuttlefish ink':'Fettuccine_al_nero_di_seppia',
             'finessenatürl':'bourbon','fix for salmon cream gratin':'Fix_for_salmon_cream_gratin',
             'fix paprika goulash gypsies':'Fix_for_paprika-goulash_gypsy','fix rahm bolognese':'Fix_for_rahm_bolognese',
             'fladen pitta':'pitta_bread','flights':'poultry','frost vegetables':'vegetables','fruity chilli sauce':'chili_sauce',
             'funny fun':'rabbit','gewurzmischung garlic oil and pepper':'garlic_oil,pepper','half beef and half pork minced fish':'pork,beef',
             'herb butter café deparis. e.g.':'Cafe_de_paris','herbs for sprinkling italian herbs':'italian herbs',
             'herbs oro di parma':'Tomaten_Passiert','hot chili sauce':' chili sauce','ital. speck':'bacon','jagdwurst whore':'turkey',
            'johannisbeergelee':'currant jelly','krustebrot':'bread','kulturlöwenzahn':'dandelion','legitimate weather forecast':'cream puff',
            'meat sausage':'sausage','mixed with the espresso':'espresso','ultivitaminsaft':'juice',
             'my dear':'Mon_cherie','neck pig':'pork','oil the dr. tomatoes':'oil tomato','oil tomatoes':'oil tomato',
            'original backin':'baking_powder','palmzucker':'palm_sugar','paprika grüneoliven':'paprika_olive','paprika olives':'paprika_olive',
             'paradiescreme':'paradise cream','paradise cream chocolate from':'paradise cream','paradise cream vanilla':'paradise cream',
             'parmesan':'parmesan_cheese','pellkartoffel':'potato','pepperoni rot':'pepperoni',
             'pickle vinegar the pickles':'gherkin','pig neck':'pork','pilzsosse':'mushroom_sauce',
             'pizzasauce':'pizza_sauce','pizzateig':'pizza_dough','pork bacon':'bacon','pork belly':'pork','porked pig':'pork',
            'powdered sugar':'sugar','prostrudel':'Prostrudel','pudding red strawberry flavor':'strawberry_pudding',
             'puten':'turkey','raz eö hanout':'Ras_el_hanout','red beefsteak tomatoes':'tomato','red fruit jelly raspberry pudding':'raspberry_pudding',
             'red chili':'chili','red pepper':'bell_pepper','red pepper':'bell_pepper','red snapper s a´':'red snapper',
            'replace the eggs with no egg powder':'egg','rippe palmin':'coco fat','rocket salad':'rucola',
             'room':'rum','room interior':'cheese','rot wood':'Madeira','rotepfefferbeeren':'pepper_berry',
            'salch':'sausage','salmon on spinach leaves from':'salmon,spinach','scene':'cream','sea-salt':'salt',
            'seasoning salt':'seasoned salt','semmelbrösel':'bread_crumbs','soup green':'vegetables','saure bohnen haribo':'candy',
             'sour cucumbers':'gherkin','soybean hl':'soy_flour','spice fire eaters from easy gourmet':'spices',
             'spice flowers':'spice','spicy':'Pikant','still additional':'olive_oil','streaky bacon or fat bacon':'bacon',
             'suckling pig leg':'pork','sweet almonds':'almond','sweet sauce':'sambal manis','the':'salt',
            'the cake stain yogurt':'yoghurt','tomato happens':'Tomaten_Passiert','tomato to taste':'Tomato_al_gusto',
            'tomato al gusto herbs':'Tomato_al_gusto','tomatosoup':'tomato_soup','tortilla plus nachocheese chips':'tortilla',
             'trip':'rice','u.pastegewürz mühlev. penny würzen':'pasta_spice','vanilla aroma liquid by dr.öter':'vanilla_liquid',
             'vanilla icecream':'vanilla_ice_cream','vanilla pod':'vanilla','vanilleback':'vanilla_backing_powder',
             'vanillenote':'vanilla','vanillepudding':'vanilla_pudding','vanillesosse':'vanilla_sauce','vanillin zucker':'vanilla_sugar',
            'waldpilz sauce':'mushroom_sauce','weintraube weiss':'grape','wheat flour':'wheat','white asparagus':'asparagus',
             'white baking chocolate':'white_chocolate','white beans':'navy_bean','white candy':'sugar','white chocolate cream penny':'white chocolate',
             'white rum':'rum','whore':'turkey','wiener tortenboden':'Viennese_cake_base','winter apple':'apple',
             'winter spices':'spices','worcestershiresauce':'Worcester_soy_sauce','würzbutter cafe de paris':'Cafe_de_paris',
             'yellow u. green peas':'peas','ziehbutter':'butter'}

In [81]:
for k,v in change_dict.items():
    for index,row in long_ingr.iterrows():
        if re.search(k,row.en_ingr)!=None:
            long_ingr.loc[index,'en_ingr']=v

In [91]:
long_ingr

,recipe_id,recipe_href,name,ingredient_name,en_ingr
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse,cream cheese
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade,white chocolate
2,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,nuss,nut
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce,mushroom_sauce
4,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur,sauce
...,...,...,...,...,...
767,51195,/rezept/451254/Raclette-Grillkartoffeln-mit-Sc...,kräuterbutter café deparis. z.b.- aus meinem m...,kräuterbutter café deparis. z.b.,Cafe_de_paris
768,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel,onion
769,92957,/rezept/370498/Krokantpudding-Kirsch-Streuselk...,vanilleback von dr. oe,vanilleback,vanilla_backing_powder
770,104968,/rezept/489228/Salat-Cedro-mit-Bueffelmozzarel...,"wer das nicht hat,nimmt eingutes salz mit kräu...",salz,salt


In [83]:
# whole delete
whole_delete=['picture paprika cracker', 'we found sauce at will']

In [84]:
long_ingr=long_ingr[~long_ingr['en_ingr'].isin(whole_delete)]

In [90]:
long_ingr.reset_index(drop=True,inplace=True)

In [86]:
# delete words
delete_words=['finesse','brown','sheet','dr.oeer','e','mixture','spanish','too','kernels','heart','hearts','s','korn ...',
'leaf','rod','minced','oeer','pee','mix','filling','dr.oeer','pure','dessertcremeerdbeergeschmack','slices',
'of','soft','some','chop','supply','dragees','bites']

In [87]:
def del_words(ingredient):
    no_words=' '.join([c for c in ingredient.split() if c not in delete_words])
    return no_words

In [88]:
long_ingr['en_ingr']=long_ingr['en_ingr'].apply(lambda x:del_words(x))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
# delete part
delete_parts=['pre-cooked to the bite','that is about 1','pre-cooked to the bite','or white wine']

In [92]:
for part in delete_parts:
    for index,row in long_ingr.iterrows():
        if re.search(part,row.en_ingr)!=None:
            long_ingr.loc[index,'en_ingr']=re.sub(part,'',row.en_ingr)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
# parse
# 'turkey',

In [94]:
for index,row in long_ingr.iterrows():
    if re.search('turkey',row.en_ingr)!=None:
        long_ingr.loc[index,'en_ingr']='turkey'

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [101]:
lemmatizer = WordNetLemmatizer()

In [102]:
long_ingr['tag']='n'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
long_ingr['ingr_lemma']=long_ingr.apply(lambda x:lemmatizer.lemmatize(x.en_ingr,x.tag),axis=1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [105]:
long_ingr.drop('tag',axis=1,inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [106]:
long_ingr_keep['en_ingr']=long_ingr_keep['ingredient_name']
long_ingr_keep['ingr_lemma']=long_ingr_keep['ingredient_name']

In [107]:
long_ingr_keep

,recipe_id,recipe_href,name,ingredient_name,en_ingr,ingr_lemma
61,60985,/rezept/480084/Baumkuchen-Petit-Fours-ergibt-c...,palmin 100 % kokosfett 1würfel,Kokosfett,Kokosfett,Kokosfett
92,60778,/rezept/479712/Alkoholische-Pralinen-mit-Grand...,mousse au chocolat%x2c 2anbieter im leb. handel,Mousse_au_Chocolat_Sterne,Mousse_au_Chocolat_Sterne,Mousse_au_Chocolat_Sterne
117,105880,/rezept/485402/Spinat-Ricotta-Tortellini.html,tomaten stückig mitkräuter von oro di parma,Tomaten_Passiert_mit_Kräuter,Tomaten_Passiert_mit_Kräuter,Tomaten_Passiert_mit_Kräuter
121,94165,/rezept/422896/Mexikanischer-Topf-auf-meiner-a...,tk mexikanischepfannengemüse z.b vom lidl,Mexikanisches_Pfannengemüse,Mexikanisches_Pfannengemüse,Mexikanisches_Pfannengemüse
146,42224,/rezept/370914/Roastbeef-und-Hueftsteak-im-Sch...,chateau de mornag wein zumsteak,Chateau_de_mornag,Chateau_de_mornag,Chateau_de_mornag
180,104794,/rezept/462642/Tintenfisch-nach-Art-der-Proven...,"kräuter- petersilie,oregano. herb de provence",Herb_de_Provence,Herb_de_Provence,Herb_de_Provence
217,36233,/rezept/433048/Cocafladen-mit-Spargel-und-Schi...,jamon iberico puro bellota,Jamon_Iberico_Puro_Bellota,Jamon_Iberico_Puro_Bellota,Jamon_Iberico_Puro_Bellota
246,56619,/rezept/288192/Valentinstorte-mit-Mon-Cheri.html,teil alkohol von mon cherioben 6 stk,Mon_Cheri,Mon_Cheri,Mon_Cheri
302,101283,/rezept/87266/Tortilla-herzhaft-gefuellt.html,mexican hot rice mix,Spicy_Mexican_Rice,Spicy_Mexican_Rice,Spicy_Mexican_Rice
322,91911,/rezept/246334/Zucchini-Karotten-Suppe.html,pesto verde von bertolli,Pesto_Verde,Pesto_Verde,Pesto_Verde


In [108]:
long_ingr_done=pd.concat([long_ingr,long_ingr_keep],axis=0,ignore_index=True)
long_ingr_done.shape

(795, 6)

In [110]:
long_ingr_done['ingr_lemma']=long_ingr_done['ingr_lemma'].apply(lambda x:x.replace(' ','_'))

In [111]:
long_ingr_done

,recipe_id,recipe_href,name,ingredient_name,en_ingr,ingr_lemma
0,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse,cream cheese,cream_cheese
1,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade,white chocolate,white_chocolate
2,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,nuss,nut,nut
3,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,waldpilz sauce von knorr,waldpilz sauce,mushroom_sauce,mushroom_sauce
4,88563,/rezept/424319/Waldpilz-Pfanne-mit-Spaetzle.html,sauce pur von knorr,sauce pur,sauce,sauce
...,...,...,...,...,...,...
790,23102,/rezept/308599/Keule-s-Partytoasts.html,tomaten passiert mitfruchtfleischstückchen von...,Tomaten_Passiert,Tomaten_Passiert,Tomaten_Passiert
791,27616,/rezept/467122/Kuchenzwerg-Philadelphiatorte-m...,götterspeise himbeer amvorabend zubereitet,Götterspeise_raspberry,Götterspeise_raspberry,Götterspeise_raspberry
792,51596,/rezept/440686/Kartoffel-Eier-Pfanne-mit-Papri...,brunch %x22sweet thai chili%x,Brunch_sweet_thai_chili,Brunch_sweet_thai_chili,Brunch_sweet_thai_chili
793,113701,/rezept/376150/Radieschen-salat-Diabolo.html,aglio olio e peperoncino,Aglio_olio_e_peperoncino,Aglio_olio_e_peperoncino,Aglio_olio_e_peperoncino


In [112]:
long_ingr_done.to_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/sub_long_ingr_data/long_long_ingr_done.csv')